# Data Collection

In [3]:
from pytube import YouTube
yt = YouTube('https://www.youtube.com/watch?v=U5fX9tbA_Tg')

In [6]:
print(yt.title)
print(yt.thumbnail_url)

Live from Space: James Zooms w/ NASA Astronauts
https://i.ytimg.com/vi/U5fX9tbA_Tg/maxresdefault.jpg


In [7]:
# streams available to download
yt.streams.all()

[<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">,
 <Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">,
 <Stream: itag="137" mime_type="video/mp4" res="1080p" fps="30fps" vcodec="avc1.640028" progressive="False" type="video">,
 <Stream: itag="248" mime_type="video/webm" res="1080p" fps="30fps" vcodec="vp9" progressive="False" type="video">,
 <Stream: itag="136" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001f" progressive="False" type="video">,
 <Stream: itag="247" mime_type="video/webm" res="720p" fps="30fps" vcodec="vp9" progressive="False" type="video">,
 <Stream: itag="135" mime_type="video/mp4" res="480p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">,
 <Stream: itag="244" mime_type="video/webm" res="480p" fps="30fps" vcodec="vp9" progressive="False" type="video">,
 <Stream:

In [60]:
import pandas as pd
videos = pd.read_csv('youtube_videos.csv')
videos.head()

,link,start,end,class,split_length
0,https://www.youtube.com/watch?v=U5fX9tbA_Tg,0:00,0:35,organic,5
1,NaN,0:35,0:39,electronic,5
2,NaN,0:40,0:47,organic,5
3,NaN,0:47,1:10,electronic,5


In [70]:
import os
from pydub import AudioSegment
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

# convert youtube time format into seconds
def get_sec(time_str):
    time_format = time_str.count(':')
    if (time_format == 1): # MM:SS
        m, s = time_str.split(':')
        return int(m) * 60 + int(s)
    else: # HH:MM:SS
        h, m, s = time_str.split(':')
        return int(h) * 3600 + int(m) * 60 + int(s)

# get filename without extension
def path_leaf(path):
    head, tail = os.path.split(path)
    return os.path.splitext(tail)[0] or os.path.basename(head)

# read csv to generate dataset
current_video = ''
dst_folder = 'youtube/'
item = 0

for index, row in videos.iterrows():
    
    # download video
    if('https://www.youtube.com/watch?v=' in str(row['link'])): # new link encountered
        current_video = row['link']
        print('parsing video', current_video)
        
        # download audio in original format
        src = YouTube(row['link']
            ).streams.filter(file_extension='mp4', only_audio=True).first( # get mp4 audio stream
            ).download(output_path=dst_folder, filename='audio_'+str(index)) # write stream to file
        print('video downloaded to', current_loc)
        
        # lossless conversion of mp4 to wav
        dst = dst_folder + path_leaf(src) + '.wav'
        sound = AudioSegment.from_file(src)
        sound.export(dst, format="wav")

    # determine time region for cuts
    splice_len = row['split_length']
    start_time = get_sec(row['start'])
    end_time = get_sec(row['end'])
    
    # make cuts by split length
    for i in range(start_time, end_time - splice_len, splice_len):
        print('clip #:', item, 'clip start:', i, 'clip end:', i + splice_len, 'video start:', start_time, 'video end:', end_time)
        ffmpeg_extract_subclip(dst, i, end_time if (i + splice_len > end_time) else (i + splice_len), targetname='youtube/'+row['class']+'/'+str(item)+'-'+str(i)+'.wav')
        item += 1

parsing video https://www.youtube.com/watch?v=U5fX9tbA_Tg
video downloaded to /Users/rramnauth/Desktop/audio-feature-extraction/youtube/audio_0.mp4
clip #: 0 clip start: 0 clip end: 5 video start: 0 video end: 35
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
clip #: 1 clip start: 5 clip end: 10 video start: 0 video end: 35
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
clip #: 2 clip start: 10 clip end: 15 video start: 0 video end: 35
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
clip #: 3 clip start: 15 clip end: 20 video start: 0 video end: 35
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
clip #: 4 clip start: 20 clip end: 25 video start: 0 video end: 35
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
clip #: 5 clip start: 25 clip end: 30 video start: 0 video end: 35
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
clip #: 6 clip start: 40 clip end: 45 v